In [ ]:
# # Create a list to store generated images
# generated_images = []
# vae_model.eval()
# image1 = torch.from_numpy(np.array(Image.fromarray(ds_train[15340][0]).resize((178, 218), Image.NEAREST))).unsqueeze(0)  # Old Babylonian
# image2 = torch.from_numpy(np.array(Image.fromarray(ds_train[3500][0]).resize((178, 218), Image.NEAREST))).unsqueeze(0)  # Middle Babylonian
# with torch.no_grad():
#     latent_vector1 = model_resume.representation(image1.unsqueeze(0))
#     latent_vector2 = model_resume.representation(image2.unsqueeze(0))

# for i in [0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1]:
#     new_tablet = i * latent_vector1 + (1 - i) * latent_vector2
#     new_tab_long = model_resume.fc3(new_tablet)
#     with torch.no_grad():
#         generated_image = model_resume.decoder(new_tab_long)
#     generated_image = generated_image[0][0].detach().cpu().numpy()  # Assuming generated image is in channel-last format (C, H, W)
#     generated_images.append((generated_image * 255).astype('uint8'))  # Convert image to uint8 and scale to 0-255

# # Save the generated images as GIF
# imageio.mimsave('generated_images.gif', generated_images, duration=0.5)

In [11]:
import numpy as np
import ipywidgets as widgets
from IPython.display import display, clear_output
from ipywidgets import widgets, Output
import torch
from PIL import Image
from era_data import TabletPeriodDataset, get_IDS
from VAE_model_tablets_class import VAE
import pandas as pd
from visualization_funcs import generate_image_from_VAE
import plotly.graph_objs as go
from ipywidgets import Image, Layout, HBox, VBox, Label
from sklearn.cluster import KMeans
from PIL import Image as PILImage
import io
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

In [12]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# IMG_DIR = 'output/images'
RUN_NAME_SUFFIX = '-masked_w_classification_loss' # ''
IMG_DIR = 'output/images_preprocessed'
LR = 5e-5
EPOCHS = 30
BATCH_SIZE = 16
SUFFIX = '-resnet50'
DATE = 'Oct2-v3'

In [13]:
IDS = get_IDS(IMG_DIR=IMG_DIR)
len(IDS)

VERSION_NAME = f'period_clf_bs{BATCH_SIZE}_lr{LR}_{EPOCHS}epochs{SUFFIX}-{len(IDS)}_samples{RUN_NAME_SUFFIX}_blurvae-conv-{DATE}'

num_classes = len(TabletPeriodDataset.PERIOD_INDICES)

class_weights = torch.load("data/class_weights_period.pt")

In [14]:
chekpoint_path = f'lightning_logs/{VERSION_NAME}/checkpoints/epoch=29-step=407516.ckpt'

In [15]:
vae_model = VAE.load_from_checkpoint(chekpoint_path,image_channels=1,z_dim=16, lr =1e-5, use_classification_loss=True, num_classes=num_classes,
            loss_type="weighted", class_weights=class_weights, device = device)

/sise/mickyfi-group/kapond/AnalysisBySynthesis/VAE_model_tablets_class.py:95: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.class_weights = torch.tensor(class_weights).to(device)


In [16]:
df_encodings_train = pd.read_csv('vae_encodings_and_data/vae_encoding_df_Oct2-v3_w_class_train.csv')
df_encodings_test = pd.read_csv('vae_encodings_and_data/vae_encoding_df_Oct2-v3_w_class_test.csv')

In [17]:
# Filter the dataframe for the current primary value
df_means = df_encodings_train.drop(["Period", "Genre", "Genre_Name"], axis = 1).groupby("Period_Name").mean().reset_index()

In [20]:
import ipywidgets as widgets
from IPython.display import display
import torch
import numpy as np
from PIL import Image
import io
from ipywidgets import Layout, HBox

# Assuming df_means is a DataFrame with Period_Name and other columns
period_names = df_means['Period_Name'].unique()

# Prepare the widgets
image_widget = widgets.Image(layout=Layout(height='200px', width='200px', border='2px solid black'))
slider = widgets.FloatSlider(value=0, min=0, max=1, step=0.1, description='Interpolation')
dropdown1 = widgets.Dropdown(options=period_names, description='Period 1', layout=Layout(width='20%'))
dropdown2 = widgets.Dropdown(options=period_names, description='Period 2', layout=Layout(width='20%'))

def get_image_from_period(period_name):
    period_data = torch.from_numpy(df_means[df_means["Period_Name"] == period_name].drop(["Period_Name"], axis=1).values[0].astype('float32'))
    return period_data

def interpolate_and_display(change):
    image1 = get_image_from_period(dropdown1.value)
    image2 = get_image_from_period(dropdown2.value)

    i = slider.value
    new_tablet = i * image1 + (1 - i) * image2
    new_tab_long = vae_model.fc3(new_tablet).unsqueeze(0)
    
    with torch.no_grad():
        generated_image = vae_model.decoder(new_tab_long)
    generated_image = generated_image[0][0].detach().cpu().numpy()  # Assuming generated image is in channel-last format (C, H, W)
    # Normalize and convert to uint8
    generated_image = (generated_image * 255).astype(np.uint8)
    # Convert to PIL image for display
    pil_img = Image.fromarray(generated_image)
    # Update the image widget
    img_byte_arr = io.BytesIO()
    pil_img.save(img_byte_arr, format='PNG')
    image_widget.value = img_byte_arr.getvalue()

# def reset_slider(*args):
#     slider.value = 0
#     update_period_images()
    
# Set initial images
interpolate_and_display({'new': 0})

# Observe changes
slider.observe(interpolate_and_display, names='value')
dropdown1.observe(interpolate_and_display, names='value')
dropdown2.observe(interpolate_and_display, names='value')

# Arrange the dropdowns horizontally
dropdowns = HBox([dropdown1, dropdown2])

# Display the widgets
display(widgets.VBox([dropdowns, slider, image_widget]))


In [34]:
import ipywidgets as widgets
from IPython.display import display
import torch
import numpy as np
from PIL import Image
import io
from ipywidgets import Layout, HBox, VBox, Button

period_names = df_means['Period_Name'].unique()

# Prepare the widgets
image_widget = widgets.Image(layout=Layout(height='200px', width='200px', border='2px solid black'))
slider = widgets.FloatSlider(value=0, min=0, max=1, step=0.1, description='Interpolation')
dropdown1 = widgets.Dropdown(options=period_names, description='Period 1', layout=Layout(width='60%'))
dropdown2 = widgets.Dropdown(options=period_names, description='Period 2', layout=Layout(width='60%'))
interpolate_button = Button(description="Interpolate")

def get_image_from_period(period_name):
    period_data = torch.from_numpy(df_means[df_means["Period_Name"] == period_name].drop(["Period_Name"], axis=1).values[0].astype('float32'))
    return period_data

def generate_image(*args):
    image1 = get_image_from_period(dropdown1.value)
    image2 = get_image_from_period(dropdown2.value)

    i = slider.value
    new_tablet = (1-i) * image1 + i * image2
    new_tab_long = vae_model.fc3(new_tablet).unsqueeze(0)
    
    with torch.no_grad():
        generated_image = vae_model.decoder(new_tab_long)
    generated_image = generated_image[0][0].detach().cpu().numpy()
    generated_image = (generated_image * 255).astype(np.uint8)
    pil_img = Image.fromarray(generated_image)
    img_byte_arr = io.BytesIO()
    pil_img.save(img_byte_arr, format='PNG')
    image_widget.value = img_byte_arr.getvalue()

def reset_slider(*args):
    slider.value = 0
    
# The interpolate_and_display function is now solely for the button click action
def interpolate_and_display(*args):
    reset_slider()
    generate_image()

# Connect the slider movement to regenerate the image without resetting
slider.observe(generate_image, names='value')

# Connect the button click event to the interpolate_and_display function without resetting the slider
interpolate_button.on_click(interpolate_and_display)

dropdowns = HBox([dropdown1, dropdown2], layout=Layout(justify_content='center'))

# Arrange the image widget to be centered
image_container = HBox([image_widget], layout=Layout(justify_content='center'))

# Combine all components into a single vertical box, keeping everything centered
controls_and_display = VBox([dropdowns, interpolate_button, slider, image_container], layout=Layout(align_items='center'))

# Display the combined layout
display(controls_and_display)
